In [1]:
# edges are encoded at 8 bit bytes. The first bit encodes flipped or not. The last 4 bits encode slice (0, 1, 2) and number within the slice (0, 1, 2, 3)

edges   = [0x00, 0x01, 0x02, 0x03, 0x04, 0x05, 0x06, 0x07, 0x08, 0x09, 0x0A, 0x0B]
corners = [0x00, 0x01, 0x02, 0x03, 0x04, 0x05, 0x06, 0x07]
cube    = [edges, corners]



In [2]:
def generate_new(members, generators, subgroup_masks):
    
    members_list = [[[edge & subgroup_masks[0] for edge in edges], [corner & subgroup_masks[1] for corner in corners] ]  for [edges, corners] in members.values()]
    
    
    candidates, new_members = dict(), dict()
    
    for name, [edges, corners] in members.items():
        candidates.update({name + n: [[edges[0x0F & edge]     ^ (edge & 0xF0)                                         for edge in edgemove], 
                                      [(corners[0x0F & corner] & 0x0F) + (((corners[0x0F & corner] & 0xF0) + (corner & 0xF0))%0xC0)    for corner in cornermove ] ]  for n, [edgemove, cornermove] in generators.items()} )
        
    for name, [edges, corners] in candidates.items():
        candidate = [[edge & subgroup_masks[0] for edge in edges], [corner & subgroup_masks[1] for corner in corners] ]
        if candidate not in members_list:            
            new_members[name] = [edges, corners]
            members_list.append(candidate)
            
    return new_members
    
    

In [3]:
# Define base permutations (clockwise quarter turns in each face - cubies numbered from 0
moves = {'R':  [[0x00, 0x01, 0x06, 0x07, 0x04, 0x05, 0x03, 0x02, 0x08, 0x09, 0x0A, 0x0B], [0x00, 0x01, 0x07, 0x06, 0x04, 0x05, 0x02, 0x03]],
         'L':  [[0x04, 0x05, 0x02, 0x03, 0x01, 0x00, 0x06, 0x07, 0x08, 0x09, 0x0A, 0x0B], [0x05, 0x04, 0x02, 0x03, 0x00, 0x01, 0x06, 0x07]],
         'F':  [[0x00, 0x01, 0x02, 0x03, 0x04, 0x09, 0x08, 0x07, 0x05, 0x06, 0x0A, 0x0B], [0x00, 0x86, 0x02, 0x84, 0x41, 0x05, 0x43, 0x07]],
         'B':  [[0x00, 0x01, 0x02, 0x03, 0x0B, 0x05, 0x06, 0x0A, 0x08, 0x09, 0x04, 0x07], [0x87, 0x01, 0x85, 0x03, 0x04, 0x40, 0x06, 0x42]],
        # 'U':  [[0x88, 0x01, 0x8B, 0x03, 0x04, 0x05, 0x06, 0x07, 0x82, 0x09, 0x0A, 0x80], [0x44, 0x01, 0x02, 0x47, 0x83, 0x05, 0x06, 0x80]],
        # 'D':  [[0x00, 0x8A, 0x02, 0x89, 0x04, 0x05, 0x06, 0x07, 0x08, 0x81, 0x83, 0x0B], [0x00, 0x45, 0x46, 0x03, 0x04, 0x82, 0x81, 0x07]]
         'UU': [[2, 1, 0, 3, 4, 5, 6, 7, 11, 9, 10, 8], [3, 1, 2, 0, 7, 5, 6, 4]],
         'DD': [[0, 3, 2, 1, 4, 5, 6, 7, 8, 10, 9, 11], [0, 2, 1, 3, 4, 6, 5, 7]]}

In [ ]:
#group 1 subgroup_masks = [0x80, 0x00]
subgroup_masks = [0x04, 0xC0]
new, members = 1, {'': cube}
while new > 0 :
    new_members = generate_new(members, moves, subgroup_masks)
    new = len(new_members)
    members.update(new_members)
    print(len(members), len(new_members))

5 4
23 18
107 84
469 362
2085 1616
9262 7177
40446 31184
162750 122304
